<a href="https://colab.research.google.com/github/grissomlin/Alpha-Data-Cleaning-Lab/blob/main/%E3%80%8C%E8%B2%A1%E5%A0%B1%E5%84%80%E8%A1%A8%E6%9D%BF%E7%9B%B8%E9%97%9C%E7%A8%8B%E5%BC%8F%E7%A2%BC%E3%80%8D_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# 🇹🇼 Taiwan Stock Price Local Refinery (2020–2025)
# 🇺🇸 台灣全市場股價本地精煉器（2020–2025）
#
# 🔍 Purpose / 目的：
#   Automatically crawl the complete list of Taiwanese securities (listed, OTC, ETFs, DRs, etc.)
#   from TWSE/TPEx official ISIN pages, then download their historical OHLCV prices via yfinance,
#   and store them in a local SQLite database with clean structure.
#
#   自動從台灣證交所／櫃買中心官方 ISIN 頁面抓取全市場有價證券清單（上市、上櫃、ETF、TDR 等），
#   透過 yfinance 下載 2020 年至今的歷史開高低收與成交量，並儲存至本地 SQLite 資料庫。
#
# 🗂️ Output / 輸出：
#   - A single SQLite file: `local_taiwan_stock.db`
#     Contains table `stock_prices` with columns: date, symbol, open, high, low, close, volume
#   - Random sample of latest trading day for quick validation
#
#   - 單一 SQLite 檔案：`local_taiwan_stock.db`
#     包含資料表 `stock_prices`，欄位：date, symbol, open, high, low, close, volume
#   - 自動抽樣最新交易日 10 筆資料供快速驗證
#
# ⚠️ Important Notes / 注意事項：
#   - This script OVERWRITES the existing DB on every run (for reproducibility).
#   - Uses raw (unadjusted) prices to preserve true OHLC structure; adjust later if needed.
#   - Single-threaded download to avoid IP ban; includes polite delays.
#   - Symbols are suffixed with .TW (listed) or .TWO (OTC/ROTC) for yfinance compatibility.
#
#   - 每次執行會覆蓋現有 DB（確保可重現性）。
#   - 使用未複權原始價格（保留真實 K 線結構），後續如需複權請另行處理。
#   - 單執行緒下載避免被封 IP，並加入禮貌性延遲。
#   - 股票代號已自動附加 .TW（上市）或 .TWO（上櫃／興櫃）以符合 yfinance 格式。
#
# ✅ Use Case / 適用場景：
#   - Building a clean local dataset for backtesting, research, or ETL pipelines.
#   - Preparing data before uploading to cloud databases (e.g., Supabase).
#
#   - 為回測、研究或 ETL 流程建立乾淨的本地資料集。
#   - 上傳至雲端資料庫（如 Supabase）前的資料準備階段。
# =============================================================================

# 1. 安裝必要套件
!pip install yfinance tqdm pandas requests lxml -q

import os, time, requests, pandas as pd, yfinance as yf
import sqlite3
from io import StringIO
from datetime import datetime
from tqdm.auto import tqdm

# ========== 核心配置：你的 7 組市場設定 ==========
url_configs = [
    {'name': 'listed', 'url': 'https://isin.twse.com.tw/isin/class_main.jsp?market=1&issuetype=1&Page=1&chklike=Y', 'suffix': '.TW'},
    {'name': 'dr', 'url': 'https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=J&industry_code=&Page=1&chklike=Y', 'suffix': '.TW'},
    {'name': 'otc', 'url': 'https://isin.twse.com.tw/isin/class_main.jsp?market=2&issuetype=4&Page=1&chklike=Y', 'suffix': '.TWO'},
    {'name': 'etf', 'url': 'https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=I&industry_code=&Page=1&chklike=Y', 'suffix': '.TW'},
    {'name': 'rotc', 'url': 'https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=E&issuetype=R&industry_code=&Page=1&chklike=Y', 'suffix': '.TWO'},
    {'name': 'tw_innovation', 'url': 'https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=C&issuetype=C&industry_code=&Page=1&chklike=Y', 'suffix': '.TW'},
    {'name': 'otc_innovation', 'url': 'https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=A&issuetype=C&industry_code=&Page=1&chklike=Y', 'suffix': '.TWO'},
]

DB_NAME = "local_taiwan_stock.db"
START_DATE = "2020-01-01"

def log(msg: str):
    print(f"{datetime.now().strftime('%H:%M:%S')}: {msg}")

# ========== 第一階段：初始化本地資料庫 ==========
if os.path.exists(DB_NAME): os.remove(DB_NAME) # 每次執行都清空重來，保證乾淨
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE stock_prices (
        date TEXT,
        symbol TEXT,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER,
        PRIMARY KEY (date, symbol)
    )
""")
conn.commit()

try:
    log("🚀 啟動本地精煉任務 (2020-2025)...")

    # 第二階段：獲取全市場清單
    all_stocks = []
    for cfg in url_configs:
        log(f"📡 抓取 {cfg['name']} 原始清單...")
        try:
            resp = requests.get(cfg['url'], timeout=15)
            df = pd.read_html(StringIO(resp.text), header=0)[0]
            for _, row in df.iterrows():
                code = str(row.get('有價證券代號', '')).strip()
                if code.isalnum() and len(code) >= 4:
                    all_stocks.append((f"{code}{cfg['suffix']}", str(row.get('有價證券名稱', '')).strip()))
        except Exception as e:
            log(f"❌ {cfg['name']} 爬取失敗: {e}")

    # 第三階段：單執行緒逐一抓取並存入 SQLite
    log(f"🚀 開始下載股價並存入本地 DB，預計處理 {len(all_stocks)} 檔...")
    success_count = 0

    for symbol, name in tqdm(all_stocks, desc="精煉進度"):
        try:
            # threads=False 確保數據不衝突，auto_adjust=False 確保四價原始正確
            data = yf.download(symbol, start=START_DATE, progress=False, threads=False, auto_adjust=False)
            if data.empty: continue

            # 處理多層索引
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)

            data = data.reset_index()
            data.columns = [c.lower() for c in data.columns]

            # 資料轉換
            data['symbol'] = symbol
            data['date'] = data['date'].dt.strftime('%Y-%m-%d')

            # 只取四價、量與日期
            final_df = data[['date', 'symbol', 'open', 'high', 'low', 'close', 'volume']]

            # 寫入 SQLite
            final_df.to_sql('stock_prices', conn, if_exists='append', index=False)
            success_count += 1

            # 每 20 檔稍微休息，防止 IP 被鎖
            if success_count % 20 == 0:
                time.sleep(0.5)

        except Exception:
            continue

    conn.commit()
    log(f"✅ 精煉完成！成功儲存 {success_count} 檔個股歷史數據。")

    # ========== 第四階段：隨機抽樣 10 筆最新交易日數據 ==========
    log("📊 正在執行數據抽樣檢查...")

    # 先找出資料庫中最新的交易日
    cursor.execute("SELECT MAX(date) FROM stock_prices")
    latest_date = cursor.fetchone()[0]

    if latest_date:
        query = f"""
            SELECT date, symbol, open, high, low, close, volume
            FROM stock_prices
            WHERE date = '{latest_date}'
            ORDER BY RANDOM()
            LIMIT 10
        """
        df_samples = pd.read_sql(query, conn)

        print("\n" + "="*80)
        print(f"📅 最新交易日 ({latest_date}) 隨機 10 筆抽樣檢查：")
        print("="*80)
        # 格式化顯示價格
        pd.options.display.float_format = '{:.2f}'.format
        print(df_samples.to_string(index=False))
        print("="*80)
        print("💡 請檢查上述 [close] 是否與看盤軟體一致。若正確，即可將 DB 轉出為 CSV。")
    else:
        print("❌ 抽樣失敗：資料庫內無有效日期數據。")

except Exception as e:
    log(f"❌ 嚴重錯誤: {e}")
finally:
    conn.close()

In [ ]:
# =============================================================================
# 📊 [Cell 0] 台灣上市公司 & 上櫃公司「月營收資料」自動化抓取工具
# =============================================================================
# 🔍 功能說明：
#   - 從公開資訊觀測站 (MOPS) 自動抓取最近 N 個月的月營收報表（含 SII/OTC/ROTC + KY 股）
#   - 支援「雙主機 fallback」：mops.twse.com.tw（舊） + mopsov.twse.com.tw（新）
#   - 自動處理編碼（Big5/UTF-8）、表格解析、欄位標準化、數值清洗、去重
#   - 輸出兩份 CSV：
#       1. 完整原始資料（含來源標記）
#       2. （可擴展）未來可用於生成 watchlist 或異常偵測
#
# ⚙️ 核心特色：
#   ✅ 民國年月 → 西元年月 自動轉換
#   ✅ 六類報表合併：上市(SII) / 上櫃(OTC) / 興櫃(ROTC) × 一般股/KY股
#   ✅ 欄位模糊匹配 + 正則標準化（容忍空格、全形、別名）
#   ✅ 數值清洗：處理「－」、「--」、「,」等非標準格式
#   ✅ 多線程加速（ThreadPoolExecutor）+ 指數退避重試機制
#   ✅ 嚴格過濾「合計行」與「無效代號」
#   ✅ 自動去重：同月同板塊同股票代號只保留第一筆
#
# 📅 時間範圍控制：
#   - 以 END_GREG_YM 為終點（西元 YYYY-MM），往回抓 MONTHS_TO_FETCH 個月
#   - 預設抓 80 個月 ≈ 6.5 年（涵蓋 2019–2025），確保包含疫情前後完整週期
#
# 📁 輸出檔案：
#   - {OUTPUT_DIR}/tw_monthly_revenue_{N}m.csv
#   - 包含關鍵 metadata 欄位：「資料年月（民國）」「資料類型（如 sii@mopsov.twse.com.tw）」
#
# ⚠️ 注意事項：
#   - MOPS 網站偶有維護或結構微調，若大規模失敗請檢查 URL 模式或 User-Agent
#   - 「去年同月增減(%)」等計算欄位由原始 HTML 提供，非本腳本計算（保留原始語意）
#   - 若需「累計營收 YoY」等衍生指標，建議後續在 Pandas 中另行計算
#   - 資料年月格式為「民國_月」（如 113_01），符合台灣財報慣例
#
# 🧪 驗證建議：
#   - 檢查輸出統計中的「覆蓋年月範圍」是否符合預期
#   - 抽查個股（如 2330、3711）確認營收數值與 MOPS 網頁一致
#   - 觀察「pages_miss」是否過高（>10% 可能需調整重試策略）
#
# 📌 作者：Data Engineering Team | 最後更新：2025 年
# =============================================================================


# 📌 [Cell 0] 抓取最近 N 個月的月營收（含舊年回溯、雙主機 fallback）→ CSV
import os, time, requests, re
import pandas as pd
from io import StringIO
from datetime import date, datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# ================== ✅ 可調整參數 ==================
MONTHS_TO_FETCH   = 80     # 2025-09 往回 75 個月 ≈ 到 2019-...（涵蓋 2020 起）
END_GREG_YM       = "2025-12"  # 若要精準到 2025-09 收斂，設定這個
MAX_WORKERS       = 6
OUTPUT_DIR        = "/content"
FIXED_OUTPUT_NAME = f"tw_monthly_revenue_{MONTHS_TO_FETCH}m.csv"
WATCHLIST_NAME    = f"tw_monthly_revenue_{MONTHS_TO_FETCH}m_watchlist.csv"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# ===================================================

print(f"🚀 開始抓取最近 {MONTHS_TO_FETCH} 個月的月營收資料...")

def greg_to_roc_year_month(ym: str):
    # "YYYY-MM" -> (roc_year, month)
    dt = datetime.strptime(ym, "%Y-%m")
    return dt.year - 1911, dt.month

def get_previous_months_roc_by_end(end_ym=END_GREG_YM, n=MONTHS_TO_FETCH):
    """依指定結束年月（西元 YYYY-MM）往回 n 個月，輸出 ROC 年月 tuple 列表。"""
    dt = datetime.strptime(end_ym, "%Y-%m")
    out = []
    y, m = dt.year, dt.month
    for i in range(n):
        yy = y
        mm = m - i
        while mm <= 0:
            yy -= 1
            mm += 12
        out.append((yy - 1911, mm))
    return out

def fetch_table_with_retry(url, max_retries=4, delay=1.3):
    """嘗試抓取 + 解析 table；失敗重試並指數退避。"""
    last_err = None
    for attempt in range(max_retries):
        try:
            r = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, timeout=30)
            r.raise_for_status()
            # 舊頁多為 big5，也有 UTF-8，試 big5 失敗再用原始
            r.encoding = 'big5'
            txt = r.text or ""
            if len(txt) < 80 or '<table' not in txt.lower():
                # 再試不指定編碼
                r2 = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, timeout=30)
                txt2 = r2.text or ""
                if len(txt2) < 80 or '<table' not in txt2.lower():
                    raise ValueError("內容過短或無 <table>")
                tables = pd.read_html(StringIO(txt2))
            else:
                tables = pd.read_html(StringIO(txt))
            if tables:
                return tables
        except Exception as e:
            last_err = e
            if attempt == max_retries - 1:
                return None
            time.sleep(delay)
            delay *= 1.6
    return None

def normalize_cols(df):
    cols = []
    for c in df.columns:
        c = str(c).replace('\u3000',' ').replace('　',' ').replace('\xa0',' ').strip()
        c = re.sub(r'\s+', ' ', c)
        cols.append(c)
    df.columns = cols
    return df

# 欄名映射（放寬）
RENAME_MAP_BASE = {
    '公司 代號':'公司代號',
    '公司代號':'公司代號',
    '證券代號':'公司代號',
    '股票代號':'公司代號',
    '公司名稱':'公司名稱',
    '公司簡稱':'公司名稱',
    '名稱':'公司名稱',
    '當月營收':'當月營收',
    '本月營收':'當月營收',
    '當月營業收入':'當月營收',
    '上月營收':'上月營收',
    '去年同月營收':'去年同月營收',
    '去年當月營收':'去年同月營收',     # 統一到「去年同月營收」
    '去年同期營收':'去年同月營收',
    '去年同月營業收入':'去年同月營收',
    '上月比較 增減(%)':'上月比較 增減(%)',
    '前期比較 增減(%)':'前期比較 增減(%)',
    '備註':'備註',
}

# 動態把「去年 同月 增減(%)」這種空格差異也併到同名
def apply_rename(df):
    df = normalize_cols(df)
    ren = {}
    for c in df.columns:
        # 捕捉「去年…同月…增減(%)」
        if re.match(r'^去年.*同月.*增減\(%\)\s*$', c):
            ren[c] = '去年同月 增減(%)'
        elif c in RENAME_MAP_BASE:
            ren[c] = RENAME_MAP_BASE[c]
    return df.rename(columns=ren)

def to_num(x):
    if pd.isna(x): return pd.NA
    x = str(x).replace(',','').replace('－','-').replace('--','').strip()
    try:
        return float(x)
    except:
        return pd.NA

def accept_df(df):
    """鬆綁驗收：要有公司代號/公司名稱/當月營收（至少兩者+當月營收）"""
    need = set(df.columns)
    ok = (('公司代號' in need or '公司名稱' in need) and ('當月營收' in need))
    return ok

def build_urls(roc_y, m):
    """同一個 ROC 年月，兩組主機輪流嘗試。"""
    paths = [
        ('sii', 't21/sii/t21sc03_{y}_{m}_0.html'),
        ('otc', 't21/otc/t21sc03_{y}_{m}_0.html'),
        ('rotc','t21/rotc/t21sc03_{y}_{m}_0.html'),
        ('sii_ky','t21/sii/t21sc03_{y}_{m}_1.html'),
        ('otc_ky','t21/otc/t21sc03_{y}_{m}_1.html'),
        ('rotc_ky','t21/rotc/t21sc03_{y}_{m}_1.html'),
    ]
    hosts = [
        'mopsov.twse.com.tw',  # 近年
        'mops.twse.com.tw',    # 舊年
    ]
    urls = []
    for h in hosts:
        for tag, pat in paths:
            urls.append((f"{tag}@{h}", f"https://{h}/nas/{pat.format(y=roc_y, m=m)}"))
    return urls

def fetch_single_month(roc_y, m):
    yymm = f"{roc_y}_{int(m):02d}"
    # 嘗試雙主機 + 六類頁面
    processed = []
    for key, url in build_urls(roc_y, m):
        tables = fetch_table_with_retry(url)
        if not tables:
            continue
        for t in tables:
            # 扁平欄名
            if isinstance(t.columns, pd.MultiIndex):
                t.columns = t.columns.get_level_values(-1)
            t = apply_rename(t)
            if not accept_df(t):
                continue
            # 清理合計列
            if '公司名稱' in t.columns:
                t = t[~t['公司名稱'].astype(str).str.contains('合計|全部', na=False)]
            if '公司代號' in t.columns:
                t['公司代號'] = t['公司代號'].astype(str).str.strip()
                t = t[t['公司代號'].str.contains(r'\d', na=False)]

            # 數值轉換
            for c in ['當月營收','上月營收','去年同月營收','上月比較 增減(%)','去年同月 增減(%)',
                      '當月累計營收','去年累計營收','前期比較 增減(%)']:
                if c in t.columns:
                    t[c] = t[c].map(to_num)

            # 補 metadata 欄
            t['資料年月'] = yymm
            t['資料類型'] = key
            processed.append(t)

        # 只要某主機+頁面抓到有效表格，就繼續嘗試其他頁面；不要過早 break，因為六類要整併
    return processed

def download_monthly_reports(roc_months, max_workers=MAX_WORKERS):
    tasks = [(y, m) for (y, m) in roc_months]
    all_tables, stats = [], {'months':len(tasks), 'pages_ok':0, 'pages_miss':0}

    def _worker(y, m):
        try:
            return fetch_single_month(y, m)
        except Exception:
            return []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = {ex.submit(_worker, y, m): (y, m) for (y, m) in tasks}
        with tqdm(total=len(futs), desc="下載月份", unit="月") as pbar:
            for fut in as_completed(futs):
                res = fut.result()
                if res:
                    all_tables.extend(res)
                    stats['pages_ok'] += len(res)
                else:
                    stats['pages_miss'] += 1
                pbar.set_postfix(stats)
                pbar.update(1)
    print("\n=== 下載統計 ===")
    for k,v in stats.items(): print(f"{k}: {v}")
    return all_tables

# --- 執行抓取 ---
roc_months = get_previous_months_roc_by_end(END_GREG_YM, MONTHS_TO_FETCH)
print("處理月份 (民國年, 月):", roc_months[:5], "...", roc_months[-5:])  # 前後各列 5 筆
print("\n開始下載…")
tables = download_monthly_reports(roc_months)

if not tables:
    raise SystemExit("❌ 抓取失敗：無任何表格")

merged = pd.concat(tables, ignore_index=True)

# 欄位排序：把資料年月/資料類型放前面
front = ['資料年月','資料類型']
cols = front + [c for c in merged.columns if c not in front]
merged = merged[cols]

# 去重（同月同板塊同代號拿第一筆）
keys = ['資料年月','資料類型','公司代號']
if all(k in merged.columns for k in keys):
    before = len(merged)
    merged = merged.drop_duplicates(subset=keys, keep='first')
    print(f"去重：{before} → {len(merged)}")

# 輸出
out_csv = os.path.join(OUTPUT_DIR, FIXED_OUTPUT_NAME)
merged.to_csv(out_csv, index=False, encoding='utf_8_sig')
print(f"\n✅ 月營收資料已儲存：{out_csv}")
print(f"📌 檔名：{FIXED_OUTPUT_NAME}")

# ===== 覆蓋範圍健檢（務必看這一段的輸出）=====
def roc_to_greg(ym):
    try:
        y, m = str(ym).split('_')
        return f"{int(y)+1911}-{int(m):02d}"
    except:
        return None

tmp = merged.copy()
tmp['年月'] = pd.to_datetime(tmp['資料年月'].map(roc_to_greg), errors='coerce').dt.to_period('M')
print("\n🧭 抓取覆蓋：", tmp['年月'].min(), "→", tmp['年月'].max())
print("🗓️ 年度分佈（筆數）：")
print(tmp['年月'].dt.year.value_counts().sort_index())
print("🔎 欄名快照：", list(merged.columns)[:16])


In [ ]:
# =============================================================================
# 🗃️ [Cell 1] 將月營收 CSV 檔案標準化並匯入本地 SQLite 資料庫
# =============================================================================
# 🔍 功能說明：
#   - 讀取由 MOPS 抓取的原始月營收 CSV（如 tw_monthly_revenue_80m.csv）
#   - 執行欄位標準化、數值清洗、類型轉換
#   - 匯入至輕量級 SQLite 資料庫（local_taiwan_stock.db）
#   - 自動建立關鍵索引以加速後續分析查詢
#
# ⚙️ 核心處理步驟：
#   ✅ 欄位映射：將中文欄名統一轉為英文 snake_case（如「當月營收」→ rev_current）
#   ✅ 數值清洗：移除千分位逗號、處理「--」「－」等無效值，轉為 float / NaN
#   ✅ 重複欄位合併：「去年當月營收」與「去年同月營收」統一為 rev_last_year
#   ✅ 安全寫入：使用 if_exists='replace' 確保每次都是最新完整資料集
#   ✅ 效能優化：建立三個關鍵索引：
#        - idx_rev_id:      stock_id（快速篩選個股）
#        - idx_rev_date:    report_month（按月份查詢）
#        - idx_yoy_high:    yoy_pct（用於高成長股篩選）
#
# 📁 輸入／輸出：
#   - 輸入：tw_monthly_revenue_80m.csv（需位於同目錄）
#   - 輸出：local_taiwan_stock.db（SQLite 檔案，含 monthly_revenue 表）
#
# ⚠️ 注意事項：
#   - 若執行時報錯「database is locked」，請關閉所有可能佔用 DB 的工具（如 VS Code SQLite 外掛、DBeaver、其他 Python 腳本）
#   - 數值欄位若原始為 '--' 或空白，會被轉為 NULL（pd.NA → SQLite NULL）
#   - 此腳本「不保留歷史版本」，每次執行都會覆蓋整個 monthly_revenue 表
#   - report_month 格式維持民國年月（如 "113_01"），與上游抓取一致，便於串接
#
# 💡 後續建議：
#   - 可搭配 stock_annual_k / stock_monthly_k 表進行「營收 vs 股價」交叉分析
#   - 若需支援西元年月查詢，可新增計算欄位或建立 VIEW
#
# 📌 作者：Data Pipeline Team | 最後更新：2025 年
# =============================================================================

import sqlite3
import pandas as pd
import os
from tqdm import tqdm

# 1. 設定路徑
csv_file = 'tw_monthly_revenue_80m.csv'
db_file = 'local_taiwan_stock.db'

if not os.path.exists(csv_file):
    print(f"❌ 找不到檔案: {csv_file}")
    exit()

print("📖 正在讀取 CSV 檔案...")
df = pd.read_csv(csv_file)

# 2. 定義欄位映射
column_mapping = {
    '資料年月': 'report_month',
    '資料類型': 'market_type',
    '公司代號': 'stock_id',
    '公司名稱': 'stock_name',
    '當月營收': 'rev_current',
    '上月營收': 'rev_last_month',
    '去年當月營收': 'rev_last_year',
    '去年同月營收': 'rev_last_year',
    '上月比較 增減(%)': 'mom_pct',
    '去年同月 增減(%)': 'yoy_pct',
    '當月累計營收': 'rev_accumulated',
    '去年累計營收': 'rev_accumulated_last_year',
    '前期比較 增減(%)': 'yoy_accumulated_pct',
    '備註': 'remark'
}

print("🧹 正在標準化欄位與資料清理...")
df.columns = [c.strip() for c in df.columns]
df = df.rename(columns=column_mapping)

# 數值清理
numeric_cols = ['rev_current', 'rev_last_month', 'rev_last_year', 'mom_pct', 'yoy_pct']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '').replace('--', '').replace('－', ''), errors='coerce')

# 3. 寫入資料庫 (帶進度條)
try:
    print(f"📦 正在寫入資料庫: {db_file} ...")
    conn = sqlite3.connect(db_file, timeout=30) # 增加 timeout 防止鎖定

    # 存入資料庫
    df.to_sql('monthly_revenue', conn, if_exists='replace', index=False)

    print("⚡ 正在建立索引 (Index) 以優化查詢速度...")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_rev_id ON monthly_revenue (stock_id)")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_rev_date ON monthly_revenue (report_month)")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_yoy_high ON monthly_revenue (yoy_pct)")

    conn.commit()
    conn.close()
    print(f"✅ 成功！已匯入 {len(df)} 筆數據。")

except sqlite3.OperationalError as e:
    print(f"❌ 錯誤：資料庫可能被其他程式佔用中。請關閉 VS Code 的 SQLite 瀏覽器或其它 Python 程式。")
    print(f"詳細錯誤訊息: {e}")

In [ ]:
# =============================================================================
# 🔍 [Cell 2] SQLite 資料庫診斷工具：檢視表結構 + 隨機抽樣資料
# =============================================================================
# 🔍 功能說明：
#   - 檢查本地 SQLite 資料庫（local_taiwan_stock.db）是否存在
#   - 列出所有資料表（table）
#   - 對每張表顯示：
#       • 欄位名稱與 SQLite 型別（TEXT / REAL / INTEGER 等）
#       • 總筆數（格式化千分位）
#       • 隨機抽取 1～3 筆實際資料（用於快速驗證內容正確性）
#
# ⚙️ 設計亮點：
#   ✅ 安全讀取：使用 pandas + PRAGMA 避免手動解析 schema
#   ✅ 防呆機制：處理空表、損壞 DB、讀取異常等邊界情況
#   ✅ 直觀輸出：表格化呈現，便於人工核對（如確認 rev_current 是否為數值）
#   ✅ 非破壞性：僅讀取，不修改任何資料
#
# 📁 輸入：
#   - local_taiwan_stock.db（預期包含 monthly_revenue、stock_monthly_k 等表）
#
# 🎯 使用情境：
#   - 驗證前一步驟（CSV 匯入）是否成功
#   - 快速確認欄位命名與資料類型是否符合分析需求
#   - 排查「為何 JOIN 查不到資料？」等問題
#   - 分享資料庫給同事時，提供自助式檢視方式
#
# ⚠️ 注意事項：
#   - 若報錯 "database disk image is malformed"，表示 DB 檔案損壞，需重新生成
#   - 隨機抽樣使用 ORDER BY RANDOM()，在大表上可能稍慢（但本腳本僅限小規模本地 DB）
#   - 欄位型別顯示的是 SQLite 實際儲存型態（非 Pandas dtype），可能為 ""（未指定）
#
# 💡 建議搭配：
#   - 執行於 CSV 匯入後、正式分析前
#   - 與「月營收抓取」和「股價聚合」流程串成完整 pipeline
#
# 📌 作者：Data Validation Team | 最後更新：2025 年
# =============================================================================


import sqlite3
import pandas as pd
import os

db_path = "local_taiwan_stock.db"

# 檢查檔案是否存在
if not os.path.exists(db_path):
    print(f"❌ 找不到資料庫檔案: {db_path}")
else:
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. 列出所有資料表
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in cursor.fetchall()]

        if not tables:
            print("📭 資料庫中沒有任何資料表。")
        else:
            print(f"📋 資料庫 '{db_path}' 中的資料表：")
            for table in tables:
                print(f"\n--- 資料表: {table} ---")

                # 2. 取得欄位資訊
                cursor.execute(f"PRAGMA table_info({table});")
                columns = cursor.fetchall()
                col_names = [col[1] for col in columns]
                col_types = [col[2] for col in columns]
                print(f"欄位名稱: {col_names}")
                print(f"欄位類型: {col_types}")

                # 3. 隨機抽樣幾筆資料（用 pandas 更方便）
                try:
                    # 先看總筆數
                    total = pd.read_sql(f"SELECT COUNT(*) AS cnt FROM {table}", conn).iloc[0]['cnt']
                    print(f"總筆數: {total:,}")

                    if total > 0:
                        # 隨機取 3 筆（若資料少於 3 筆則全取）
                        sample_size = min(3, total)
                        # 使用 RANDOM() 抽樣
                        df_sample = pd.read_sql(
                            f"SELECT * FROM {table} ORDER BY RANDOM() LIMIT {sample_size}",
                            conn
                        )
                        print("隨機樣本:")
                        print(df_sample.to_string(index=False))
                    else:
                        print("（無資料）")
                except Exception as e:
                    print(f"⚠️ 讀取資料失敗: {e}")

        conn.close()
        print("\n✅ 資料庫檢查完成。")

    except sqlite3.DatabaseError as e:
        print(f"❌ SQLite 資料庫損壞: {e}")
        print("請重新上傳完好的 .db 檔案！")

In [ ]:
# =============================================================================
# 🔍 [Cell 2] SQLite 資料庫診斷工具：檢視表結構 + 隨機抽樣資料
# =============================================================================
# 🔍 功能說明：
#   - 檢查本地 SQLite 資料庫（local_taiwan_stock.db）是否存在
#   - 列出所有資料表（table）
#   - 對每張表顯示：
#       • 欄位名稱與 SQLite 型別（TEXT / REAL / INTEGER 等）
#       • 總筆數（格式化千分位）
#       • 隨機抽取 1～3 筆實際資料（用於快速驗證內容正確性）
#
# ⚙️ 設計亮點：
#   ✅ 安全讀取：使用 pandas + PRAGMA 避免手動解析 schema
#   ✅ 防呆機制：處理空表、損壞 DB、讀取異常等邊界情況
#   ✅ 直觀輸出：表格化呈現，便於人工核對（如確認 rev_current 是否為數值）
#   ✅ 非破壞性：僅讀取，不修改任何資料
#
# 📁 輸入：
#   - local_taiwan_stock.db（預期包含 monthly_revenue、stock_monthly_k 等表）
#
# 🎯 使用情境：
#   - 驗證前一步驟（CSV 匯入）是否成功
#   - 快速確認欄位命名與資料類型是否符合分析需求
#   - 排查「為何 JOIN 查不到資料？」等問題
#   - 分享資料庫給同事時，提供自助式檢視方式
#
# ⚠️ 注意事項：
#   - 若報錯 "database disk image is malformed"，表示 DB 檔案損壞，需重新生成
#   - 隨機抽樣使用 ORDER BY RANDOM()，在大表上可能稍慢（但本腳本僅限小規模本地 DB）
#   - 欄位型別顯示的是 SQLite 實際儲存型態（非 Pandas dtype），可能為 ""（未指定）
#
# 💡 建議搭配：
#   - 執行於 CSV 匯入後、正式分析前
#   - 與「月營收抓取」和「股價聚合」流程串成完整 pipeline
#
# 📌 作者：Data Validation Team | 最後更新：2025 年
# =============================================================================

import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text

# --- 配置區 ---
DB_PASSWORD = "Supabase Service Role 密碼"          # 你的 Supabase Service Role 密碼
PROJECT_REF = "專案 ID（postgres.<PROJECT_REF>）"     # 專案 ID（postgres.<PROJECT_REF>）
POOLER_HOST = "aws-1-ap-southeast-1.pooler.supabase.com"
connection_string = f"postgresql://postgres.{PROJECT_REF}:{DB_PASSWORD}@{POOLER_HOST}:5432/postgres"

print("🔄 開始執行智慧同步：只上傳新增的股價資料")

# 1. 從本地 SQLite 取得最新交易日
print("🔍 步驟 1/3：讀取本地最新交易日期...")
conn_sqlite = sqlite3.connect("local_taiwan_stock.db")
local_max_date = pd.read_sql("SELECT MAX(date) AS max_date FROM stock_prices", conn_sqlite).iloc[0]['max_date']
conn_sqlite.close()
print(f"📅 本地最新交易日: {local_max_date}")

# 2. 連接 Supabase，取得遠端最新交易日
engine = create_engine(connection_string)
remote_max_date = None

try:
    print("🔍 步驟 2/3：查詢 Supabase 最新交易日期...")
    with engine.connect() as conn:
        try:
            result = conn.execute(text("SELECT MAX(date) AS max_date FROM stock_prices"))
            row = result.fetchone()
            remote_max_date = row[0] if row and row[0] else None
            print(f"☁️ Supabase 最新交易日: {remote_max_date or '（無資料）'}")
        except Exception as e:
            if 'relation "stock_prices" does not exist' in str(e):
                print("🆕 Supabase 尚未建立 stock_prices 表（首次同步）")
                remote_max_date = None
            else:
                raise e

    # 3. 決定要上傳哪些資料
    if remote_max_date is None:
        print("📤 判斷：首次同步 → 上傳全部本地資料")
        conn_sqlite = sqlite3.connect("local_taiwan_stock.db")
        upload_df = pd.read_sql("SELECT * FROM stock_prices", conn_sqlite)
        conn_sqlite.close()
    elif local_max_date <= remote_max_date:
        print("✅ 判斷：Supabase 已是最新，無需上傳")
        upload_df = None
    else:
        print(f"📤 判斷：有新資料！上傳日期 > '{remote_max_date}' 的記錄")
        conn_sqlite = sqlite3.connect("local_taiwan_stock.db")
        upload_df = pd.read_sql(
            "SELECT * FROM stock_prices WHERE date > ?",
            conn_sqlite,
            params=[remote_max_date]
        )
        conn_sqlite.close()
        print(f"📊 共找到 {len(upload_df):,} 筆新資料")

    # 4. 執行上傳（如果有資料）
    if upload_df is not None and len(upload_df) > 0:
        print(f"🚀 開始上傳 {len(upload_df):,} 筆資料到 Supabase...")
        upload_df.to_sql(
            name='stock_prices',
            con=engine,
            if_exists='append',
            index=False,
            chunksize=10000,
            method='multi'
        )
        print("✅ 上傳成功！")
    elif upload_df is not None:
        print("ℹ️ 無新資料需要上傳。")

except Exception as e:
    print(f"❌ 同步失敗: {e}")

finally:
    engine.dispose()
    print("🔌 資料庫連線已關閉。")

In [ ]:
# =============================================================================
# 📈 [Cell 3] 台股「還原股價」全頻率聚合清洗流程（週/月/年 K）
# =============================================================================
# 🔍 功能說明：
#   - 從 Supabase 資料庫取得所有曾發布月營收的股票代號（含上市 .TW / 上櫃 .TWO）
#   - 使用 yfinance 抓取近 5 年日頻資料（含 Adj Close）
#   - 執行「專業級股價清洗」：偵測並剔除「乒乓極端震盪」（如減資、併購異常波動）
#   - 聚合生成三張標準化 K 線表：
#       • stock_annual_k：年度開/收盤（用於計算真實年報酬）
#       • stock_monthly_k：月度開/收盤（對齊民國年月格式，可串接營收）
#       • stock_weekly_k：週線收盤（用於公告時序事件研究）
#
# ⚙️ 核心技術亮點：
#   ✅ 使用 Adj Close 計算漲跌幅 → 自動處理除權息、減資、併股等事件
#   ✅ 「乒乓過濾」機制：若連續兩日漲跌幅 >40% 且方向相反，視為異常並剔除
#        （有效排除如 8476 台境等減資股的虛假暴漲）
#   ✅ 月 K 對齊「民國年月」格式（e.g., "113_01"），與 monthly_revenue 表無縫 JOIN
#   ✅ 年 K 不直接用 yfinance 年頻，而是由日頻聚合 → 確保 Adj Close 邏輯一致
#   ✅ 週 K 仍使用 yfinance 的 1wk 頻率（符合交易所週定義），再經同樣清洗
#
# 🌐 資料來源與連線：
#   - 股票清單：Supabase PostgreSQL (monthly_revenue 表)
#   - 股價來源：Yahoo Finance (yfinance)，涵蓋 .TW / .TWO
#   - 寫回目標：同一 Supabase 資料庫（覆蓋 stock_*_k 表）
#
# ⚠️ 注意事項：
#   - yfinance 可能偶有缺失（尤其冷門櫃買股），腳本會自動跳過失敗標的
#   - PINGPONG_THRESHOLD = 0.40 是經驗值，可依需求調整（太低會誤刪，太高會漏濾）
#   - 此流程「不保留原始未清洗股價」，僅輸出乾淨後的還原價格
#   - 若需「成交量」或「最高/最低價」，可擴展 agg 函數（目前聚焦開/收盤）
#
# 💡 分析價值：
#   - 真實年報酬 = year_close / year_open - 1（已校正公司行為）
#   - 可與月營收 YoY 搭配，計算「營收成長 vs 股價表現」散佈圖
#   - 週線可用於檢驗「營收公告後 1 週是否跑贏大盤」
#
# 📌 作者：Quant Research Team | 最後更新：2025 年
# =============================================================================


import yfinance as yf
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import urllib.parse
from datetime import datetime

# --- 1. 配置區 ---
DB_PASSWORD = "Supabase Service Role 密碼"          # 你的 Supabase Service Role 密碼
PROJECT_REF = "專案 ID（postgres.<PROJECT_REF>）"     # 專案 ID（postgres.<PROJECT_REF>）
POOLER_HOST = "aws-1-ap-southeast-1.pooler.supabase.com"
connection_string = f"postgresql://postgres.{PROJECT_REF}:{urllib.parse.quote_plus(DB_PASSWORD)}@{POOLER_HOST}:5432/postgres"
engine = create_engine(connection_string)

# 專業清洗門檻
PINGPONG_THRESHOLD = 0.40

# --- 2. 專業清洗函數 ---
def clean_k_data(df):
    if df.empty: return df
    df = df.sort_values('Date').reset_index(drop=True)

    # 使用 Adj Close 來計算真實漲跌幅，避開減資/除權息誤判
    df['pct_change'] = df['Adj Close'].pct_change()

    mask_pingpong = pd.Series(False, index=df.index)
    for i in range(1, len(df)-1):
        prev = df.loc[i, 'pct_change']
        nxt = df.loc[i+1, 'pct_change']
        if pd.notna(prev) and pd.notna(nxt):
            # 偵測極端震盪 (乒乓)
            if abs(prev) > PINGPONG_THRESHOLD and abs(nxt) > PINGPONG_THRESHOLD and prev * nxt < 0:
                mask_pingpong.iloc[i] = True
                mask_pingpong.iloc[i+1] = True

    return df[~mask_pingpong].dropna(subset=['Adj Close']).copy()

# --- 3. 執行抓取 ---
with engine.connect() as conn:
    # 這裡我們從營收表抓所有出現過的股票，確保完整性
    symbols = pd.read_sql("SELECT DISTINCT stock_id FROM monthly_revenue", conn)['stock_id'].tolist()
    # 加上台股後綴
    symbols = [f"{s}.TW" if len(s)==4 else f"{s}.TWO" for s in symbols]

print(f"🚀 開始執行『還原股價』全頻率清洗 (周/月/年 K)...")

all_weekly, all_monthly, all_annual = [], [], []

for i, sym in enumerate(symbols):
    try:
        # 下載數據 (包含 Adj Close)
        # 年K改由日K聚合以獲得最精確的 Adj Close 轉換
        raw_data = yf.download(sym, period="5y", interval="1d", progress=False)

        if raw_data.empty: continue

        # 1. 基礎清洗
        df_clean = clean_k_data(raw_data.reset_index())
        df_clean['symbol'] = sym

        # 2. 處理年 K (年度報酬最關鍵：使用還原股價)
        df_clean['year_str'] = df_clean['Date'].dt.year.astype(str)
        annual_summary = df_clean.groupby('year_str').agg(
            year_open=('Adj Close', 'first'),
            year_close=('Adj Close', 'last')
        ).reset_index()
        annual_summary.columns = ['year', 'year_open', 'year_close']
        annual_summary['symbol'] = sym
        all_annual.append(annual_summary)

        # 3. 處理月 K (對齊營收月份)
        df_clean['report_month'] = df_clean['Date'].apply(lambda x: f"{x.year-1911}_{x.month:02d}")
        monthly_summary = df_clean.groupby('report_month').agg(
            m_open=('Adj Close', 'first'),
            m_close=('Adj Close', 'last')
        ).reset_index()
        monthly_summary['symbol'] = sym
        all_monthly.append(monthly_summary)

        # 4. 處理周 K (用於公告時序研究)
        # 周線我們重新抓取 1wk 頻率以對齊標準周定義
        raw_w = yf.download(sym, period="5y", interval="1wk", progress=False)
        if not raw_w.empty:
            df_w = clean_k_data(raw_w.reset_index())
            df_w['symbol'] = sym
            df_w = df_w[['Date', 'symbol', 'Adj Close']] # 簡化，主要看還原後的收盤
            df_w.columns = ['date', 'symbol', 'w_close']
            all_weekly.append(df_w)

        if (i+1) % 50 == 0:
            print(f"✅ 已處理 {i+1} 檔 ({(i+1)/len(symbols)*100:.1f}%)")

    except Exception: continue

# --- 4. 覆蓋資料庫 ---
if all_annual:
    pd.concat(all_annual).to_sql('stock_annual_k', engine, if_exists='replace', index=False)
if all_monthly:
    pd.concat(all_monthly).to_sql('stock_monthly_k', engine, if_exists='replace', index=False)
if all_weekly:
    pd.concat(all_weekly).to_sql('stock_weekly_k', engine, if_exists='replace', index=False)

print("\n✨ 數據大清洗完成！台境 (8476) 等減資股的漲幅已回歸真實。")

In [ ]:
# =============================================================================
# 📌 目標：建立「周K線資料表」stock_weekly_k（從 stock_daily_k 聚合）
# 📌 輸出欄位：date, symbol, w_open, w_close, w_high, w_low
# 📌 注意：不包含 volume；週末定義為「每週五」（若無交易則取該週最後交易日）
# =============================================================================

# 1️⃣ 安裝必要套件（僅首次執行需要）
!pip install sqlalchemy psycopg2-binary pandas -q

# 2️⃣ 匯入模組
import pandas as pd
from sqlalchemy import create_engine, text
import urllib.parse

# =============================================================================
# 🔐 Supabase 連線設定（請確認正確）
# =============================================================================
DB_PASSWORD = "Supabase Service Role 密碼"          # 你的 Supabase Service Role 密碼
PROJECT_REF = "專案 ID（postgres.<PROJECT_REF>）"     # 專案 ID（postgres.<PROJECT_REF>）
POOLER_HOST = "aws-1-ap-southeast-1.pooler.supabase.com"

encoded_password = urllib.parse.quote_plus(DB_PASSWORD)
connection_string = (
    f"postgresql://postgres.{PROJECT_REF}:{encoded_password}"
    f"@{POOLER_HOST}:5432/postgres?sslmode=require"
)
engine = create_engine(connection_string)

# =============================================================================
# 🔍 第一步：確認來源表 stock_daily_k 是否存在
# =============================================================================
def table_exists(table_name):
    with engine.connect() as conn:
        result = conn.execute(text("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_schema = 'public' AND table_name = :name
            );
        """), {"name": table_name}).scalar()
        return result

if not table_exists("stock_daily_k"):
    print("❌ 錯誤：來源表 'stock_daily_k' 不存在！")
    print("   請先確保有日K資料表，欄位包含：date, symbol, open, high, low, close")
else:
    print("✅ 找到來源表 stock_daily_k，開始建立周K...")

    # =============================================================================
    # 🛠️ 第二步：建立 stock_weekly_k（作為 Table，非 View）
    # 使用 CREATE TABLE AS 語法一次性寫入所有周K資料
    # =============================================================================
    create_weekly_sql = """
    CREATE TABLE IF NOT EXISTS stock_weekly_k AS
    WITH weekly_groups AS (
        SELECT
            symbol,
            date,
            open,
            high,
            low,
            close,
            -- 將日期歸屬到「當週五」（ISO 週：週一=1, 週日=7）
            -- DATE_TRUNC('week', ...) 回傳週一，+6天 → 週日；但我們要週五 → +4
            DATE_TRUNC('week', date)::DATE + 4 AS week_end
        FROM stock_daily_k
        WHERE date IS NOT NULL
          AND symbol IS NOT NULL
    ),
    weekly_agg AS (
        SELECT
            symbol,
            week_end AS date,
            FIRST_VALUE(open) OVER w AS w_open,
            LAST_VALUE(close) OVER w AS w_close,
            MAX(high) OVER w AS w_high,
            MIN(low) OVER w AS w_low
        FROM weekly_groups
        WINDOW w AS (
            PARTITION BY symbol, week_end
            ORDER BY date
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        )
    )
    SELECT DISTINCT
        date,
        symbol,
        w_open,
        w_close,
        w_high,
        w_low
    FROM weekly_agg
    ORDER BY symbol, date;
    """

    try:
        with engine.begin() as conn:
            # 如果已存在，先刪除（避免重複資料）
            conn.execute(text("DROP TABLE IF EXISTS stock_weekly_k;"))
            conn.execute(text(create_weekly_sql))
        print("✨ 成功建立資料表：stock_weekly_k")

        # =============================================================================
        # 🔎 第三步：顯示前 5 筆驗證
        # =============================================================================
        with engine.connect() as conn:
            result = conn.execute(text("SELECT * FROM stock_weekly_k LIMIT 5;"))
            df = pd.DataFrame(result.fetchall(), columns=result.keys())
        print("\n📊 驗證結果（前 5 筆）:")
        print(df)

    except Exception as e:
        print(f"❌ 建立失敗: {e}")
        raise

# =============================================================================
# 💡 說明：
# - 每週以「週五」為代表日（即使當天無交易，也會用該週最後交易日）
# - w_open = 該週第一筆日K的開盤價
# - w_close = 該週最後筆日K的收盤價
# - w_high / w_low = 該週最高/最低價
# - 若未來有新日K資料，需手動重新執行此腳本更新周K
# =============================================================================

In [ ]:
# =============================================================================
# 📌 目標：在 Supabase 中建立「年K線視圖」stock_annual_k
# 📌 輸入來源：stock_weekly_k（周K資料表）
# 📌 輸出欄位：symbol, year, first/last trade date, year_open/close/high/low/volume
# 📌 注意：若 stock_weekly_k 沒有 w_volume，會自動設為 NULL
# =============================================================================

# 1️⃣ 安裝必要套件（僅首次執行需要）
!pip install sqlalchemy psycopg2-binary pandas -q

# 2️⃣ 匯入所需模組
import pandas as pd
from sqlalchemy import create_engine, text
import urllib.parse

# =============================================================================
# 🔐 資料庫連線設定（請確認密碼與專案 ID 正確）
# =============================================================================
DB_PASSWORD = "Supabase Service Role 密碼"          # 你的 Supabase Service Role 密碼
PROJECT_REF = "專案 ID（postgres.<PROJECT_REF>）"     # 專案 ID（postgres.<PROJECT_REF>）
POOLER_HOST = "aws-1-ap-southeast-1.pooler.supabase.com"

# URL 編碼密碼（處理 '+' 等特殊字元）
encoded_password = urllib.parse.quote_plus(DB_PASSWORD)
connection_string = (
    f"postgresql://postgres.{PROJECT_REF}:{encoded_password}"
    f"@{POOLER_HOST}:5432/postgres?sslmode=require"
)

# 建立 SQLAlchemy 引擎
engine = create_engine(connection_string)

# =============================================================================
# 🔍 第一步：檢查 stock_weekly_k 是否有 w_volume 欄位
# =============================================================================
def check_column_exists(table_name, column_name):
    with engine.connect() as conn:
        query = text("""
            SELECT EXISTS (
                SELECT 1 FROM information_schema.columns
                WHERE table_name = :table AND column_name = :col
            );
        """)
        return conn.execute(query, {"table": table_name, "col": column_name}).scalar()

has_volume = check_column_exists("stock_weekly_k", "w_volume")
print(f"✅ stock_weekly_k 是否包含 w_volume 欄位？{'是' if has_volume else '否'}")

# =============================================================================
# 🛠️ 第二步：生成 CREATE VIEW SQL 語法（根據是否有 volume 動態調整）
# =============================================================================
if has_volume:
    volume_expr = "SUM(w_volume) AS year_volume"
else:
    volume_expr = "NULL::NUMERIC AS year_volume  -- 原始資料無成交量欄位"

create_view_sql = f"""
CREATE OR REPLACE VIEW stock_annual_k AS
SELECT DISTINCT
    symbol,
    EXTRACT(YEAR FROM date)::TEXT AS year,
    MIN(date) OVER yr AS first_trade_date,
    MAX(date) OVER yr AS last_trade_date,
    FIRST_VALUE(w_open) OVER yr AS year_open,
    LAST_VALUE(w_close) OVER yr AS year_close,
    MAX(w_high) OVER yr AS year_high,
    MIN(w_low) OVER yr AS year_low,
    {volume_expr}
FROM stock_weekly_k
WINDOW yr AS (
    PARTITION BY symbol, EXTRACT(YEAR FROM date)
    ORDER BY date
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
);
"""

# =============================================================================
# ⚙️ 第三步：執行 CREATE VIEW
# =============================================================================
try:
    with engine.begin() as conn:  # 自動 commit 或 rollback
        conn.execute(text(create_view_sql))
    print("\n✨ 成功建立或更新 View: stock_annual_k")
except Exception as e:
    print(f"\n❌ 建立 View 失敗: {e}")
    raise

# =============================================================================
# 🔎 第四步：查詢前 5 筆資料驗證結果
# =============================================================================
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT * FROM stock_annual_k LIMIT 5;"))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\n📊 驗證結果（stock_annual_k 前 5 筆）:")
    print(df)
except Exception as e:
    print(f"\n⚠️ 查詢驗證失敗（可能因無資料）: {e}")

# =============================================================================
# 💡 使用說明：
# 1. 此 View 會自動從 stock_weekly_k 聚合出年度 K 線
# 2. year_open = 該年第一筆周K的開盤價
# 3. year_close = 該年最後筆周K的收盤價
# 4. year_high/low = 該年所有周K的最高/最低價
# 5. 若無 w_volume，year_volume 將為 NULL
# =============================================================================